<a href="https://colab.research.google.com/github/idarwati/Health-Data-Analytics-Portfolio/blob/Day-4/H4_Geospatial_Health_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Install library peta dulu (biasanya belum ada di Google Colab)
!pip install folium

import folium
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import numpy as np

# ==========================================
# 1. GENERATE DATA SPASIAL DUMMY (MAKASSAR)
# ==========================================
np.random.seed(42)
n_pasien = 200

# Koordinat Pusat (Misal: RS Mata di Makassar)
rs_coords = [-5.147665, 119.432732] # Titik tengah kota

# Generate 200 Pasien tersebar acak di sekitar RS (Radius ~5-10km)
# Kita pakai "Distribusi Normal" agar pasien menumpuk di pusat kota dan menipis di pinggir
lat_pasien = rs_coords[0] + np.random.normal(0, 0.03, n_pasien)
lon_pasien = rs_coords[1] + np.random.normal(0, 0.03, n_pasien)

# Status Penyakit (Merah=Glaukoma Berat, Hijau=Ringan)
# Logika: Pasien yang jauh (pinggiran) cenderung datang terlambat (Berat)
status_penyakit = []
for lat, lon in zip(lat_pasien, lon_pasien):
    jarak = np.sqrt((lat - rs_coords[0])**2 + (lon - rs_coords[1])**2)
    if jarak > 0.04: # Jika jauh
        status_penyakit.append('Berat (Terlambat)')
    else:
        status_penyakit.append(np.random.choice(['Ringan', 'Berat'], p=[0.7, 0.3]))

df_map = pd.DataFrame({
    'Latitude': lat_pasien,
    'Longitude': lon_pasien,
    'Status': status_penyakit
})

# ==========================================
# 2. MEMBUAT PETA INTERAKTIF
# ==========================================

# A. Buat Peta Dasar (Canvas)
peta = folium.Map(location=rs_coords, zoom_start=13, tiles="CartoDB positron")

# B. Tandai Lokasi Rumah Sakit (Ikon Besar Biru)
folium.Marker(
    location=rs_coords,
    popup="<b>RS Mata Pusat</b>",
    icon=folium.Icon(color='blue', icon='plus', prefix='fa')
).add_to(peta)

# C. Plot Titik Pasien
# Merah = Kasus Berat, Hijau = Kasus Ringan
for index, row in df_map.iterrows():
    warna = 'red' if row['Status'] == 'Berat (Terlambat)' else 'green'
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=warna,
        fill=True,
        fill_color=warna,
        fill_opacity=0.7,
        popup=f"Status: {row['Status']}"
    ).add_to(peta)

# D. Tambahkan Heatmap (Peta Panas)
# Ini menunjukkan konsentrasi/kerumunan pasien
data_heat = df_map[['Latitude', 'Longitude']].values
HeatMap(data_heat, radius=15).add_to(peta)

# E. Tampilkan Peta
print("Peta Sebaran Pasien Glaukoma (Merah=Berat, Hijau=Ringan):")
peta

Peta Sebaran Pasien Glaukoma (Merah=Berat, Hijau=Ringan):
